<a href="https://colab.research.google.com/github/SamGRosen/samgrosen.github.io/blob/master/files/code/matchgame.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from collections import defaultdict

import numpy as np
import scipy.sparse
import scipy.sparse.linalg

def remove_last_row_column(sparse_matrix):
  num_states = sparse_matrix.shape[0]
  row_mask = np.ones(sparse_matrix.shape[0], dtype=bool)
  row_mask[num_states-1] = False
  col_mask = np.ones(sparse_matrix.shape[1], dtype=bool)
  col_mask[num_states-1] = False
  return sparse_matrix[row_mask][:,col_mask]

def get_markov_chain(number_of_pairs, entire_matrix=False, extra_info=False):
  queue = [(0, 2 * number_of_pairs)]

  adjacency_list = {}

  while queue:
    curr_state = queue.pop()
    alpha, beta = curr_state
  
    if alpha < 0 or beta < 0:
      continue
  
    next_states = {}
    if alpha > beta or (alpha + beta) % 2 == 1: # Know more knowns than unknowns or there is a duplicate in the flipped cards
      next_states[(alpha - 1, beta)] = 1
    elif alpha == 0 and beta == 0:
      next_states[(0, 0)] = 1
    elif alpha == beta:
      next_states[(alpha - 1, beta - 1)] = 1
    else:
      next_states[(alpha - 1, beta - 1)] = alpha / beta
      next_states[(alpha + 2, beta - 2)] = (1 - alpha/beta) * (1 - (alpha+1)/(beta-1))
      next_states[(alpha + 1, beta - 2)] = (1 - alpha/beta) * (alpha / (beta - 1))
      next_states[(alpha, beta - 2)]     = (1 - alpha/beta) * 1 / (beta - 1)

    # exclude calculating transitions for states with 0 probability to reach
    queue.extend(state for state, probability in next_states.items() if state not in adjacency_list and probability > 0)

    adjacency_list[curr_state] = next_states

  for state, neighbors in adjacency_list.items(): # Clear out not possible states
    adjacency_list[state] = {k: v for k, v in neighbors.items() if v > 0}

  num_states = len(adjacency_list)
  markov_chain = scipy.sparse.lil_matrix((num_states, num_states))

  # Build enumeration for states
  sorted_keys = sorted(list(adjacency_list.keys()), key=lambda pair: -sum(pair))
  keys_to_index = {key: index for index, key in enumerate(sorted_keys)}

  for state, neighbors in adjacency_list.items():
    adjacency_list[state] = {k: v for k, v in neighbors.items() if v > 1e-10}

    for destination, probability in neighbors.items():
      markov_chain[(keys_to_index[state], keys_to_index[destination])] = probability
  
  if not entire_matrix: # Get block matrix for fundamental matrix calculation
    to_return = remove_last_row_column(markov_chain)
  else:
    to_return = scipy.sparse.csr_matrix(markov_chain)

  if extra_info:
    return to_return, { "adjacency_list": adjacency_list, "keys_to_index": keys_to_index, "sorted_keys": sorted_keys }
  return to_return
  

# N^2 + 1 states
three_case = get_markov_chain(3, entire_matrix=True)


In [ ]:
def get_case_info(n_pairs):
  markov_matrix = get_markov_chain(n_pairs, entire_matrix=True)
  Q_matrix = remove_last_row_column(markov_matrix)

  # Solve with the fundamental matrix
  expected_number_of_turns = scipy.sparse.linalg.spsolve( scipy.sparse.eye(Q_matrix.shape[0]) - Q_matrix, np.ones(Q_matrix.shape[0]))[0]
  expected_number_of_failures = expected_number_of_turns - n_pairs

  starting_point = np.zeros(markov_matrix.shape[0])
  starting_point[0] = 1

  first_chance_to_win = markov_matrix.T ** n_pairs
  prob_vector = first_chance_to_win @ starting_point
  chance_to_finish_with_failures = [prob_vector[-1]]

  # Almost all the processing time is calculating the failure distribution
  for number_of_failures in range(n_pairs):
    next_prob_vector = markov_matrix.T @ prob_vector
    chance_to_finish_with_failures.append(next_prob_vector[-1] - prob_vector[-1])
    prob_vector = next_prob_vector
  
  return np.array(chance_to_finish_with_failures)


get_case_info(19)


array([1.21939404e-22, 2.78021842e-19, 2.68221572e-16, 1.41849116e-13,
       4.46687950e-11, 8.54907374e-09, 9.78191679e-07, 6.37166308e-05,
       2.16683350e-03, 3.36708317e-02, 2.01656540e-01, 4.12461113e-01,
       2.81803227e-01, 6.36489224e-02, 4.44655751e-03, 8.10055350e-05,
       2.66017169e-07, 6.57639498e-11, 0.00000000e+00, 0.00000000e+00])

In [ ]:
def export_markov_chain_data(up_to_n):
  first_appears = {0: set( ((0,0),) )}
  parents_on_level = {}

  # Build reverse lookup for our d3 visualizations
  reverse_lookup = defaultdict(dict)
  for i in range(1, up_to_n + 1):
    sparse_mat, extra_info = get_markov_chain(i, entire_matrix=True, extra_info=True)
    adj_list = extra_info["adjacency_list"]
    sorted_keys = extra_info["sorted_keys"]
    possible_states = set(adj_list.keys())

    for j in range(i):
      possible_states -= first_appears[j]

    first_appears[i] = possible_states

    # https://stackoverflow.com/a/4319087/10013298
    cx = scipy.sparse.coo_matrix(sparse_mat)    
    for i,j,p in zip(cx.row, cx.col, cx.data):
        reverse_lookup[sorted_keys[j]][sorted_keys[i]] = p


  to_export = []
  for level, states in first_appears.items():
    for state in states:
      parents = reverse_lookup[state]
      parent_and_probs = list(parents.items()) # ensure order is maintained, likely not a problem with python 3.6+
      to_export.append({
          "id": str(state), # javascript needs pairs as strings
          "firstAppears": level,
          "parentIds": [str(key) for key, _ in parent_and_probs],
          "probabilities": [prob for _, prob in parent_and_probs]
      })
  
  for item in to_export:
    if item["id"] == "(0, 0)":
      item["parentIds"].remove("(0, 0)") # no loops
      item["probabilities"].pop() # all values are 1.0
      break
  
  return to_export

# Print in json format
print(str(export_markov_chain_data(10)).replace("'", '"'))

[{"id": "(0, 0)", "firstAppears": 0, "parentIds": ["(0, 2)", "(1, 1)"], "probabilities": [1.0, 1.0]}, {"id": "(0, 2)", "firstAppears": 1, "parentIds": ["(0, 4)", "(1, 3)"], "probabilities": [0.3333333333333333, 0.3333333333333333]}, {"id": "(0, 4)", "firstAppears": 2, "parentIds": ["(0, 6)", "(1, 5)"], "probabilities": [0.2, 0.2]}, {"id": "(2, 2)", "firstAppears": 2, "parentIds": ["(0, 4)", "(2, 4)", "(3, 2)", "(3, 3)"], "probabilities": [0.6666666666666667, 0.16666666666666666, 1.0, 1.0]}, {"id": "(1, 1)", "firstAppears": 2, "parentIds": ["(2, 2)", "(1, 3)", "(2, 1)"], "probabilities": [1.0, 0.33333333333333337, 1.0]}, {"id": "(3, 2)", "firstAppears": 3, "parentIds": ["(2, 4)"], "probabilities": [0.3333333333333333]}, {"id": "(1, 3)", "firstAppears": 3, "parentIds": ["(2, 4)", "(1, 5)", "(2, 3)"], "probabilities": [0.5, 0.2, 1.0]}, {"id": "(2, 1)", "firstAppears": 3, "parentIds": ["(1, 3)"], "probabilities": [0.33333333333333337]}, {"id": "(0, 6)", "firstAppears": 3, "parentIds": ["(0

In [ ]:
def export_markov_chain_winning_data(n_pairs):
  data = []
  for n in range(1, n_pairs + 1):
    info = get_case_info(n)

    pdf = info[info > 0.0] # We do floating point comparison as we want to keep incredibly small positive values
    support = np.arange(0, len(pdf), 1)

    expected_number_of_failures = np.sum(pdf * support)
    variance = np.mean((support - expected_number_of_failures) ** 2)

    data.append(dict(
        pdf = list(pdf),
        expectation = expected_number_of_failures
     ))
    
  return data

print(str(export_markov_chain_winning_data(10)).replace("'", '"'))



[{"pdf": [1.0], "expectation": 0.0}, {"pdf": [0.3333333333333333, 0.6666666666666667], "expectation": 0.6666666666666667}, {"pdf": [0.06666666666666667, 0.5333333333333334, 0.3999999999999999], "expectation": 1.3333333333333333}, {"pdf": [0.009523809523809523, 0.19047619047619052, 0.6666666666666669, 0.1333333333333333], "expectation": 1.923809523809524}, {"pdf": [0.001058201058201058, 0.04232804232804233, 0.3915343915343915, 0.5333333333333333, 0.031746031746031744], "expectation": 2.5523809523809526}, {"pdf": [9.62000962000962e-05, 0.006734006734006735, 0.12794612794612797, 0.5670033670033671, 0.2922558922558922, 0.00596440596440595], "expectation": 3.1624819624819622}, {"pdf": [7.4000074000074e-06, 0.0008288008288008289, 0.027972027972027982, 0.28344988344988353, 0.5675213675213677, 0.11928811928811933, 0.0009324009324008786], "expectation": 3.77924297924298}, {"pdf": [4.933338266671599e-07, 8.288008288008289e-05, 0.004516964516964518, 0.08651299317965988, 0.45908659241992583, 0.410